In [ ]:
import requests
import json
import csv
import pandas as pd
response = requests.get('https://sensor-api.example.com/data', verify=True) ## we are not taking live data for now this code can be improved more and use for better in live data
json_data = response.json()
df = pd.DataFrame(json_data)
df.to_csv('output.csv', index=False)

MODEL-A

In [8]:
import pandas as pd
import xgboost as xgb
import json
import requests
import io
import numpy as np # Import numpy for calculating activity magnitude

# Base class for animal monitoring
class AnimalMonitoring:
    def __init__(self, model_paths):
        self.models = {}
        # Load models only if paths are provided and exist
        if model_paths:
            for disease, path in model_paths.items():
                try:
                    self.models[disease] = xgb.Booster(model_file=path)
                except xgb.core.XGBoostError as e:
                    print(f"Warning: Could not load model for {disease} from {path}. Error: {e}")
                    self.models[disease] = None # Or handle as appropriate


    def load_data(self, source, data_path=None, bom_data_path=None):
        if source == 'web':
            # This part might need adaptation if web data also includes BOM-like info
            return self._load_from_web()
        elif source == 'dataset':
            if data_path:
                # If data_path is a list, load each file separately
                if isinstance(data_path, list):
                    loaded_data = []
                    for path in data_path:
                        data_df = self._load_from_dataset(path)
                        # Load and merge BOM data if provided
                        if bom_data_path:
                            bom_df = pd.read_csv(bom_data_path)
                            # Assuming 'timestamp' is the common column
                            # Convert timestamp to datetime for merging - Correct format for BOM
                            data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], format='%d-%m-%Y', errors='coerce')
                            bom_df['timestamp'] = pd.to_datetime(bom_df['timestamp'], format='%Y-%m-%d', errors='coerce') # Corrected format here
                            # Perform merge, assuming a many-to-one relationship (multiple animal readings per BOM timestamp)
                            merged_data = pd.merge(data_df, bom_df, on='timestamp', how='left')
                            loaded_data.append(merged_data)
                        else:
                             loaded_data.append(data_df)
                    return loaded_data # Return a list of DataFrames
                else:
                    # Load a single file
                    combined_data = self._load_from_dataset(data_path)
                    # Load and merge BOM data if provided
                    if bom_data_path:
                        bom_df = pd.read_csv(bom_data_path)
                        # Assuming 'timestamp' is the common column
                        # Convert timestamp to datetime for merging - Correct format for BOM
                        combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'], format='%d-%m-%Y', errors='coerce')
                        bom_df['timestamp'] = pd.to_datetime(bom_df['timestamp'], format='%Y-%m-%d', errors='coerce') # Corrected format here
                        # Perform merge, assuming a many-to-one relationship (multiple animal readings per BOM timestamp)
                        merged_data = pd.merge(combined_data, bom_df, on='timestamp', how='left')
                        return merged_data
                    else:
                        return combined_data
            else:
                raise ValueError("data_path must be provided for 'dataset' source")
        else:
            raise ValueError("Invalid source. Choose 'web' or 'dataset'")


    def _load_from_web(self):
        response = requests.get('https://sensor-api.example.com/data', verify=True)
        json_data = response.json()
        return pd.DataFrame(json_data)

    def _load_from_dataset(self, data_path):
        # Assuming the dataset is in CSV format
        return pd.read_csv(data_path)

    def convert_json_to_csv(self, data, output_filename='output.csv'):
        # This method might need to be adapted or moved to subclasses depending on data structure
        if isinstance(data, pd.DataFrame):
            data.to_csv(output_filename, index=False)
        else:
             # Handle other data types if necessary
             pass


    def prepare_features(self, data):
        # Implement chicken-specific feature engineering here
        # Calculate 'activity' from accelerometer columns if they exist
        if all(col in data.columns for col in ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']):
            data['activity'] = np.sqrt(data['accelerometer_x']**2 + data['accelerometer_y']**2 + data['accelerometer_z']**2)
        elif 'activity' not in data.columns:
             print("Warning: Accelerometer columns not found, and 'activity' column is missing. Cannot create activity feature.")
             data['activity'] = 0 # Or handle appropriately

        # Use 'temperature (°C)' from BOM and individual humidity/SpO2 if available, plus other BOM columns
        # If 'temperature (°C)' is not available, use 'temperature_y' from BOM, or 'temperature_x' from individual
        temperature_col_name = None
        if 'temperature (°C)' in data.columns:
            temperature_col_name = 'temperature (°C)'
        elif 'temperature_y' in data.columns:
            temperature_col_name = 'temperature_y'
        elif 'temperature_x' in data.columns:
            temperature_col_name = 'temperature_x'
        else:
             print("Warning: No temperature column found.")
             data['temperature'] = 0 # Or handle appropriately
             temperature_col_name = 'temperature' # Use the dummy column


        humidity_col = 'humidity' if 'humidity' in data.columns else None
        if not humidity_col:
             print("Warning: No humidity column found.")
             data['humidity'] = 0 # Or handle appropriately
             humidity_col = 'humidity' # Use the dummy column

        # Standardize temperature column name
        if temperature_col_name and temperature_col_name != 'temperature':
            if temperature_col_name in data.columns:
                data['temperature'] = data[temperature_col_name]
                data = data.drop(columns=[temperature_col_name])
            else:
                 # This case should ideally not happen if temperature_col_name was set based on existing columns
                 print(f"Error: Selected temperature column '{temperature_col_name}' not found in data columns after merge.")
                 data['temperature'] = 0 # Fallback to dummy column

        required_cols = ['activity', 'temperature', humidity_col, 'CO2 ppm', 'CO ppm', 'NH3 ppm', 'PM2.5']
        # Filter required_cols to only include columns that are now in data after potential dummy creation
        required_cols = [col for col in required_cols if col in data.columns]


        if not all(col in data.columns for col in required_cols):
            missing_cols = [col for col in required_cols if col not in data.columns]
            raise ValueError(f"Missing required columns for feature preparation: {missing_cols}. Available columns after initial processing: {data.columns.tolist()}")

        features = data[required_cols]
        return features


    def categorize_risk(self, predictions):
        risk_levels = {}
        for disease, preds in predictions.items():
            if preds is not None: # Check if predictions are available for this disease
                # Implement chicken-specific risk categorization based on disease and prediction scores
                risk_levels[disease] = ['high' if p > 0.7 else 'medium' if p > 0.4 else 'low' for p in preds]
            else:
                # Determine the expected length of risk_levels based on the input data length
                # This assumes predictions is a dictionary where values are lists/arrays of the same length
                # This logic might need adjustment if processing multiple files individually
                # For now, keeping it as is, assuming predictions is a dictionary per file in the new structure
                # If predictions is a list of dictionaries, this needs to be handled in the calling method
                pass # This method will now process predictions for a single DataFrame at a time
        return risk_levels

    def train_models(self, training_data_paths, bom_data_path=None):
        """Trains models using provided dataset paths and optionally BOM data."""
        # Load training data - this will now return a list of DataFrames if data_path is a list
        training_data_list = self.load_data(source='dataset', data_path=training_data_paths, bom_data_path=bom_data_path)

        # Concatenate all training data DataFrames into a single DataFrame for training
        if isinstance(training_data_list, list):
            training_data = pd.concat(training_data_list, ignore_index=True)
        else:
            training_data = training_data_list


        print(f"Training data columns after merging BOM: {training_data.columns.tolist()}") # Print column names for debugging

        training_features = self.prepare_features(training_data)


        # Assuming a 'disease_status' column exists in the training data for each disease
        # In a real scenario, you'd need target variables for each disease
        # For demonstration, we'll create dummy target variables based on activity and temperature
        for disease in self.models.keys():
            # Dummy target: higher risk if activity is low and temp is high
            # Ensure the target variable name is consistent
            target_column = f'{disease.replace(" ", "_").lower()}_status'
            # Check if required columns for dummy target creation exist
            # Use 'temperature' from merged data (likely from BOM)
            # Check for 'temperature (°C)' first, then other temperature columns if needed
            temperature_col_for_target = None
            if 'temperature (°C)' in training_data.columns:
                 temperature_col_for_target = 'temperature (°C)'
            elif 'temperature_y' in training_data.columns:
                 temperature_col_for_target = 'temperature_y'
            elif 'temperature_x' in training_data.columns:
                 temperature_col_for_target = 'temperature_x'
            elif 'temperature' in training_data.columns: # Check for standardized 'temperature' column
                 temperature_col_for_target = 'temperature'


            if 'activity' not in training_data.columns or temperature_col_for_target is None:
                 print(f"Warning: Cannot create dummy target for {disease}. Missing 'activity' or a suitable temperature column.")
                 target = pd.Series([0] * len(training_data)) # Create a dummy target of zeros
            else:
                # Ensure the length of the target matches the length of the training features
                if len(training_data) != len(training_features):
                    print(f"Warning: Length of training data ({len(training_data)}) does not match length of training features ({len(training_features)}). Skipping dummy target creation for {disease}.")
                    target = pd.Series([0] * len(training_features)) # Create a dummy target of zeros with feature length
                else:
                    training_data[target_column] = ((training_data['activity'] < 0.5) & (training_data[temperature_col_for_target] > 30)).astype(int)
                    target = training_data[target_column]


            # Check if training_features is empty
            if training_features.empty:
                 print(f"Warning: training_features is empty for {disease}. Skipping model training.")
                 continue

            # Check if target is empty or contains only NaNs
            if target.empty or target.isnull().all():
                 print(f"Warning: Target variable is empty or contains only NaNs for {disease}. Skipping model training.")
                 continue


            dmatrix = xgb.DMatrix(training_features, label=target)

            # Train a new model for each disease
            trained_model = xgb.train({}, dmatrix, num_boost_round=10) # Simple training
            self.trained_models[disease] = trained_model
            # Optionally save the trained model: trained_model.save_model(self.models[disease])
            print(f"Trained model for {disease}")


    def predict_with_trained_models(self, data_paths, bom_data_path=None):
        """Predicts risk using the models trained within this instance, optionally merging BOM data."""
        # Load data - this will now return a list of DataFrames if data_paths is a list
        prediction_data_list = self.load_data(source='dataset', data_path=data_paths, bom_data_path=bom_data_path)

        all_predictions = []
        all_risk_levels = []
        all_data_merged = [] # Store merged data for each file

        # Ensure prediction_data_list is always a list for consistent processing
        if not isinstance(prediction_data_list, list):
             prediction_data_list = [prediction_data_list]


        for prediction_data_merged in prediction_data_list:
            features = self.prepare_features(prediction_data_merged)
            predictions_single_file = {}
            risk_levels_single_file = {}

            for disease, model in self.trained_models.items():
                 # Check if model was trained successfully
                 if model:
                     # Check if features is empty
                     if features.empty:
                         print(f"Warning: features is empty for {disease} in a file. Skipping prediction.")
                         predictions_single_file[disease] = None
                         risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction
                         continue

                     dmatrix = xgb.DMatrix(features)
                     predictions_single_file[disease] = model.predict(dmatrix)
                     # Categorize risk using the instance's categorize_risk method
                     risk_levels_single_file[disease] = self.categorize_risk({disease: predictions_single_file[disease]})[disease]
                 else:
                     predictions_single_file[disease] = None
                     risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction

            all_predictions.append(predictions_single_file)
            all_risk_levels.append(risk_levels_single_file)
            all_data_merged.append(prediction_data_merged) # Append merged data for this file

        return all_predictions, all_risk_levels, all_data_merged # Return lists of predictions, risk levels, and data


    def convert_results_to_json(self, data, predictions, risk_levels, output_filename='results.json'):
        results = []
        # Ensure data is a list of DataFrames when processing individual files
        if not isinstance(data, list):
             data = [data]
             # Adjust predictions and risk_levels structure to match if data was not a list initially
             # This part might need more sophisticated handling depending on how predictions are returned
             # For now, assuming predictions and risk_levels are also lists if data is a list
             if not isinstance(predictions, list):
                 predictions = [predictions]
             if not isinstance(risk_levels, list):
                 risk_levels = [risk_levels]


        for i, df in enumerate(data):
            # Use filename or a unique identifier if available in the DataFrame
            # For now, using index as a placeholder for animal identifier
            animal_id = f'animal_{i+1}' # Simple identifier based on file order

            for index, row in df.iterrows():
                 record_id = f'{animal_id}_record_{index}' # Unique ID for each record
                 record_results = {'record_id': record_id, 'animal_id': animal_id, 'diseases': {}} # Added animal_id to record level
                 for disease in predictions[i]: # Iterate through predictions for the current DataFrame
                     prediction_value = predictions[i][disease][index] if predictions[i][disease] is not None and index < len(predictions[i][disease]) else None
                     risk_level_value = risk_levels[i][disease][index] if risk_levels[i][disease] is not None and index < len(risk_levels[i][disease]) else "Unknown"

                     record_results['diseases'][disease] = {
                         'prediction': float(prediction_value) if prediction_value is not None else None,
                         'risk_level': risk_level_value
                     }
                 results.append(record_results)

        with open(output_filename, 'w') as f:
            json.dump(results, f, indent=4)


    def run_monitoring(self, source, data_path=None, bom_data_path=None):
        data = self.load_data(source, data_path, bom_data_path)
        features = self.prepare_features(data)
        predictions = self.predict_disease_risk(features)
        risk_levels = self.categorize_risk(predictions)
        self.convert_results_to_json(data, predictions, risk_levels)


# Subclass for Chicken Monitoring
class ChickenMonitoring(AnimalMonitoring):
    def __init__(self, model_paths):
        super().__init__(model_paths)
        self.trained_models = {} # Store trained models if training within the class

    def prepare_features(self, data):
        # Implement chicken-specific feature engineering here
        # Calculate 'activity' from accelerometer columns if they exist
        if all(col in data.columns for col in ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']):
            data['activity'] = np.sqrt(data['accelerometer_x']**2 + data['accelerometer_y']**2 + data['accelerometer_z']**2)
        elif 'activity' not in data.columns:
             print("Warning: Accelerometer columns not found, and 'activity' column is missing. Cannot create activity feature.")
             data['activity'] = 0 # Or handle appropriately

        # Use 'temperature (°C)' from BOM and individual humidity/SpO2 if available, plus other BOM columns
        # If 'temperature (°C)' is not available, use 'temperature_y' from BOM, or 'temperature_x' from individual
        temperature_col_name = None
        if 'temperature (°C)' in data.columns:
            temperature_col_name = 'temperature (°C)'
        elif 'temperature_y' in data.columns:
            temperature_col_name = 'temperature_y'
        elif 'temperature_x' in data.columns:
            temperature_col_name = 'temperature_x'
        else:
             print("Warning: No temperature column found.")
             data['temperature'] = 0 # Or handle appropriately
             temperature_col_name = 'temperature' # Use the dummy column


        humidity_col = 'humidity' if 'humidity' in data.columns else None
        if not humidity_col:
             print("Warning: No humidity column found.")
             data['humidity'] = 0 # Or handle appropriately
             humidity_col = 'humidity' # Use the dummy column

        # Standardize temperature column name
        if temperature_col_name and temperature_col_name != 'temperature':
            if temperature_col_name in data.columns:
                data['temperature'] = data[temperature_col_name]
                data = data.drop(columns=[temperature_col_name])
            else:
                 # This case should ideally not happen if temperature_col_name was set based on existing columns
                 print(f"Error: Selected temperature column '{temperature_col_name}' not found in data columns after merge.")
                 data['temperature'] = 0 # Fallback to dummy column

        required_cols = ['activity', 'temperature', humidity_col, 'CO2 ppm', 'CO ppm', 'NH3 ppm', 'PM2.5']
        # Filter required_cols to only include columns that are now in data after potential dummy creation
        required_cols = [col for col in required_cols if col in data.columns]


        if not all(col in data.columns for col in required_cols):
            missing_cols = [col for col in required_cols if col not in data.columns]
            raise ValueError(f"Missing required columns for feature preparation: {missing_cols}. Available columns after initial processing: {data.columns.tolist()}")

        features = data[required_cols]
        return features


    def categorize_risk(self, predictions):
        risk_levels = {}
        for disease, preds in predictions.items():
            if preds is not None: # Check if predictions are available for this disease
                # Implement chicken-specific risk categorization based on disease and prediction scores
                risk_levels[disease] = ['high' if p > 0.7 else 'medium' if p > 0.4 else 'low' for p in preds]
            else:
                # Determine the expected length of risk_levels based on the input data length
                # This assumes predictions is a dictionary where values are lists/arrays of the same length
                # This logic might need adjustment if processing multiple files individually
                # For now, keeping it as is, assuming predictions is a dictionary per file in the new structure
                # If predictions is a list of dictionaries, this needs to be handled in the calling method
                pass # This method will now process predictions for a single DataFrame at a time
        return risk_levels

    def train_models(self, training_data_paths, bom_data_path=None):
        """Trains models using provided dataset paths and optionally BOM data."""
        # Load training data - this will now return a list of DataFrames if data_path is a list
        training_data_list = self.load_data(source='dataset', data_path=training_data_paths, bom_data_path=bom_data_path)

        # Concatenate all training data DataFrames into a single DataFrame for training
        if isinstance(training_data_list, list):
            training_data = pd.concat(training_data_list, ignore_index=True)
        else:
            training_data = training_data_list


        print(f"Training data columns after merging BOM: {training_data.columns.tolist()}") # Print column names for debugging

        training_features = self.prepare_features(training_data)


        # Assuming a 'disease_status' column exists in the training data for each disease
        # In a real scenario, you'd need target variables for each disease
        # For demonstration, we'll create dummy target variables based on activity and temperature
        for disease in self.models.keys():
            # Dummy target: higher risk if activity is low and temp is high
            # Ensure the target variable name is consistent
            target_column = f'{disease.replace(" ", "_").lower()}_status'
            # Check if required columns for dummy target creation exist
            # Use 'temperature' from merged data (likely from BOM)
            # Check for 'temperature (°C)' first, then other temperature columns if needed
            temperature_col_for_target = None
            if 'temperature (°C)' in training_data.columns:
                 temperature_col_for_target = 'temperature (°C)'
            elif 'temperature_y' in training_data.columns:
                 temperature_col_for_target = 'temperature_y'
            elif 'temperature_x' in training_data.columns:
                 temperature_col_for_target = 'temperature_x'
            elif 'temperature' in training_data.columns: # Check for standardized 'temperature' column
                 temperature_col_for_target = 'temperature'


            if 'activity' not in training_data.columns or temperature_col_for_target is None:
                 print(f"Warning: Cannot create dummy target for {disease}. Missing 'activity' or a suitable temperature column.")
                 target = pd.Series([0] * len(training_data)) # Create a dummy target of zeros
            else:
                # Ensure the length of the target matches the length of the training features
                if len(training_data) != len(training_features):
                    print(f"Warning: Length of training data ({len(training_data)}) does not match length of training features ({len(training_features)}). Skipping dummy target creation for {disease}.")
                    target = pd.Series([0] * len(training_features)) # Create a dummy target of zeros with feature length
                else:
                    training_data[target_column] = ((training_data['activity'] < 0.5) & (training_data[temperature_col_for_target] > 30)).astype(int)
                    target = training_data[target_column]


            # Check if training_features is empty
            if training_features.empty:
                 print(f"Warning: training_features is empty for {disease}. Skipping model training.")
                 continue

            # Check if target is empty or contains only NaNs
            if target.empty or target.isnull().all():
                 print(f"Warning: Target variable is empty or contains only NaNs for {disease}. Skipping model training.")
                 continue


            dmatrix = xgb.DMatrix(training_features, label=target)

            # Train a new model for each disease
            trained_model = xgb.train({}, dmatrix, num_boost_round=10) # Simple training
            self.trained_models[disease] = trained_model
            # Optionally save the trained model: trained_model.save_model(self.models[disease])
            print(f"Trained model for {disease}")


    def predict_with_trained_models(self, data_paths, bom_data_path=None):
        """Predicts risk using the models trained within this instance, optionally merging BOM data."""
        # Load data - this will now return a list of DataFrames if data_paths is a list
        prediction_data_list = self.load_data(source='dataset', data_path=data_paths, bom_data_path=bom_data_path)

        all_predictions = []
        all_risk_levels = []
        all_data_merged = [] # Store merged data for each file

        # Ensure prediction_data_list is always a list for consistent processing
        if not isinstance(prediction_data_list, list):
             prediction_data_list = [prediction_data_list]


        for prediction_data_merged in prediction_data_list:
            features = self.prepare_features(prediction_data_merged)
            predictions_single_file = {}
            risk_levels_single_file = {}

            for disease, model in self.trained_models.items():
                 # Check if model was trained successfully
                 if model:
                     # Check if features is empty
                     if features.empty:
                         print(f"Warning: features is empty for {disease} in a file. Skipping prediction.")
                         predictions_single_file[disease] = None
                         risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction
                         continue

                     dmatrix = xgb.DMatrix(features)
                     predictions_single_file[disease] = model.predict(dmatrix)
                     # Categorize risk using the instance's categorize_risk method
                     risk_levels_single_file[disease] = self.categorize_risk({disease: predictions_single_file[disease]})[disease]
                 else:
                     predictions_single_file[disease] = None
                     risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction

            all_predictions.append(predictions_single_file)
            all_risk_levels.append(risk_levels_single_file)
            all_data_merged.append(prediction_data_merged) # Append merged data for this file

        return all_predictions, all_risk_levels, all_data_merged # Return lists of predictions, risk levels, and data


    def convert_results_to_json(self, data, predictions, risk_levels, output_filename='results.json'):
        results = []
        # Ensure data is a list of DataFrames when processing individual files
        if not isinstance(data, list):
             data = [data]
             # Adjust predictions and risk_levels structure to match if data was not a list initially
             # This part might need more sophisticated handling depending on how predictions are returned
             # For now, assuming predictions and risk_levels are also lists if data is a list
             if not isinstance(predictions, list):
                 predictions = [predictions]
             if not isinstance(risk_levels, list):
                 risk_levels = [risk_levels]


        for i, df in enumerate(data):
            # Use filename or a unique identifier if available in the DataFrame
            # For now, using index as a placeholder for animal identifier
            animal_id = f'animal_{i+1}' # Simple identifier based on file order

            for index, row in df.iterrows():
                 record_id = f'{animal_id}_record_{index}' # Unique ID for each record
                 record_results = {'record_id': record_id, 'animal_id': animal_id, 'diseases': {}} # Added animal_id to record level
                 for disease in predictions[i]: # Iterate through predictions for the current DataFrame
                     prediction_value = predictions[i][disease][index] if predictions[i][disease] is not None and index < len(predictions[i][disease]) else None
                     risk_level_value = risk_levels[i][disease][index] if risk_levels[i][disease] is not None and index < len(risk_levels[i][disease]) else "Unknown"

                     record_results['diseases'][disease] = {
                         'prediction': float(prediction_value) if prediction_value is not None else None,
                         'risk_level': risk_level_value
                     }
                 results.append(record_results)

        with open(output_filename, 'w') as f:
            json.dump(results, f, indent=4)


    def run_monitoring(self, source, data_path=None, bom_data_path=None):
        data = self.load_data(source, data_path, bom_data_path)
        features = self.prepare_features(data)
        predictions = self.predict_disease_risk(features)
        risk_levels = self.categorize_risk(predictions)
        self.convert_results_to_json(data, predictions, risk_levels)


# Subclass for Chicken Monitoring
class ChickenMonitoring(AnimalMonitoring):
    def __init__(self, model_paths):
        super().__init__(model_paths)
        self.trained_models = {} # Store trained models if training within the class

    def prepare_features(self, data):
        # Implement chicken-specific feature engineering here
        # Calculate 'activity' from accelerometer columns if they exist
        if all(col in data.columns for col in ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']):
            data['activity'] = np.sqrt(data['accelerometer_x']**2 + data['accelerometer_y']**2 + data['accelerometer_z']**2)
        elif 'activity' not in data.columns:
             print("Warning: Accelerometer columns not found, and 'activity' column is missing. Cannot create activity feature.")
             data['activity'] = 0 # Or handle appropriately

        # Use 'temperature (°C)' from BOM and individual humidity/SpO2 if available, plus other BOM columns
        # If 'temperature (°C)' is not available, use 'temperature_y' from BOM, or 'temperature_x' from individual
        temperature_col_name = None
        if 'temperature (°C)' in data.columns:
            temperature_col_name = 'temperature (°C)'
        elif 'temperature_y' in data.columns:
            temperature_col_name = 'temperature_y'
        elif 'temperature_x' in data.columns:
            temperature_col_name = 'temperature_x'
        else:
             print("Warning: No temperature column found.")
             data['temperature'] = 0 # Or handle appropriately
             temperature_col_name = 'temperature' # Use the dummy column


        humidity_col = 'humidity' if 'humidity' in data.columns else None
        if not humidity_col:
             print("Warning: No humidity column found.")
             data['humidity'] = 0 # Or handle appropriately
             humidity_col = 'humidity' # Use the dummy column

        # Standardize temperature column name
        if temperature_col_name and temperature_col_name != 'temperature':
            if temperature_col_name in data.columns:
                data['temperature'] = data[temperature_col_name]
                data = data.drop(columns=[temperature_col_name])
            else:
                 # This case should ideally not happen if temperature_col_name was set based on existing columns
                 print(f"Error: Selected temperature column '{temperature_col_name}' not found in data columns after merge.")
                 data['temperature'] = 0 # Fallback to dummy column

        required_cols = ['activity', 'temperature', humidity_col, 'CO2 ppm', 'CO ppm', 'NH3 ppm', 'PM2.5']
        # Filter required_cols to only include columns that are now in data after potential dummy creation
        required_cols = [col for col in required_cols if col in data.columns]


        if not all(col in data.columns for col in required_cols):
            missing_cols = [col for col in required_cols if col not in data.columns]
            raise ValueError(f"Missing required columns for feature preparation: {missing_cols}. Available columns after initial processing: {data.columns.tolist()}")

        features = data[required_cols]
        return features


    def categorize_risk(self, predictions):
        risk_levels = {}
        for disease, preds in predictions.items():
            if preds is not None: # Check if predictions are available for this disease
                # Implement chicken-specific risk categorization based on disease and prediction scores
                risk_levels[disease] = ['high' if p > 0.7 else 'medium' if p > 0.4 else 'low' for p in preds]
            else:
                # Determine the expected length of risk_levels based on the input data length
                # This assumes predictions is a dictionary where values are lists/arrays of the same length
                # This logic might need adjustment if processing multiple files individually
                # For now, keeping it as is, assuming predictions is a dictionary per file in the new structure
                # If predictions is a list of dictionaries, this needs to be handled in the calling method
                pass # This method will now process predictions for a single DataFrame at a time
        return risk_levels

    def train_models(self, training_data_paths, bom_data_path=None):
        """Trains models using provided dataset paths and optionally BOM data."""
        # Load training data - this will now return a list of DataFrames if data_path is a list
        training_data_list = self.load_data(source='dataset', data_path=training_data_paths, bom_data_path=bom_data_path)

        # Concatenate all training data DataFrames into a single DataFrame for training
        if isinstance(training_data_list, list):
            training_data = pd.concat(training_data_list, ignore_index=True)
        else:
            training_data = training_data_list


        print(f"Training data columns after merging BOM: {training_data.columns.tolist()}") # Print column names for debugging

        training_features = self.prepare_features(training_data)


        # Assuming a 'disease_status' column exists in the training data for each disease
        # In a real scenario, you'd need target variables for each disease
        # For demonstration, we'll create dummy target variables based on activity and temperature
        for disease in self.models.keys():
            # Dummy target: higher risk if activity is low and temp is high
            # Ensure the target variable name is consistent
            target_column = f'{disease.replace(" ", "_").lower()}_status'
            # Check if required columns for dummy target creation exist
            # Use 'temperature' from merged data (likely from BOM)
            # Check for 'temperature (°C)' first, then other temperature columns if needed
            temperature_col_for_target = None
            if 'temperature (°C)' in training_data.columns:
                 temperature_col_for_target = 'temperature (°C)'
            elif 'temperature_y' in training_data.columns:
                 temperature_col_for_target = 'temperature_y'
            elif 'temperature_x' in training_data.columns:
                 temperature_col_for_target = 'temperature_x'
            elif 'temperature' in training_data.columns: # Check for standardized 'temperature' column
                 temperature_col_for_target = 'temperature'


            if 'activity' not in training_data.columns or temperature_col_for_target is None:
                 print(f"Warning: Cannot create dummy target for {disease}. Missing 'activity' or a suitable temperature column.")
                 target = pd.Series([0] * len(training_data)) # Create a dummy target of zeros
            else:
                # Ensure the length of the target matches the length of the training features
                if len(training_data) != len(training_features):
                    print(f"Warning: Length of training data ({len(training_data)}) does not match length of training features ({len(training_features)}). Skipping dummy target creation for {disease}.")
                    target = pd.Series([0] * len(training_features)) # Create a dummy target of zeros with feature length
                else:
                    training_data[target_column] = ((training_data['activity'] < 0.5) & (training_data[temperature_col_for_target] > 30)).astype(int)
                    target = training_data[target_column]


            # Check if training_features is empty
            if training_features.empty:
                 print(f"Warning: training_features is empty for {disease}. Skipping model training.")
                 continue

            # Check if target is empty or contains only NaNs
            if target.empty or target.isnull().all():
                 print(f"Warning: Target variable is empty or contains only NaNs for {disease}. Skipping model training.")
                 continue


            dmatrix = xgb.DMatrix(training_features, label=target)

            # Train a new model for each disease
            trained_model = xgb.train({}, dmatrix, num_boost_round=10) # Simple training
            self.trained_models[disease] = trained_model
            # Optionally save the trained model: trained_model.save_model(self.models[disease])
            print(f"Trained model for {disease}")


    def predict_with_trained_models(self, data_paths, bom_data_path=None):
        """Predicts risk using the models trained within this instance, optionally merging BOM data."""
        # Load data - this will now return a list of DataFrames if data_paths is a list
        prediction_data_list = self.load_data(source='dataset', data_path=data_paths, bom_data_path=bom_data_path)

        all_predictions = []
        all_risk_levels = []
        all_data_merged = [] # Store merged data for each file

        # Ensure prediction_data_list is always a list for consistent processing
        if not isinstance(prediction_data_list, list):
             prediction_data_list = [prediction_data_list]


        for prediction_data_merged in prediction_data_list:
            features = self.prepare_features(prediction_data_merged)
            predictions_single_file = {}
            risk_levels_single_file = {}

            for disease, model in self.trained_models.items():
                 # Check if model was trained successfully
                 if model:
                     # Check if features is empty
                     if features.empty:
                         print(f"Warning: features is empty for {disease} in a file. Skipping prediction.")
                         predictions_single_file[disease] = None
                         risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction
                         continue

                     dmatrix = xgb.DMatrix(features)
                     predictions_single_file[disease] = model.predict(dmatrix)
                     # Categorize risk using the instance's categorize_risk method
                     risk_levels_single_file[disease] = self.categorize_risk({disease: predictions_single_file[disease]})[disease]
                 else:
                     predictions_single_file[disease] = None
                     risk_levels_single_file[disease] = ["Unknown"] * len(prediction_data_merged) # Fill with "Unknown" if no prediction

            all_predictions.append(predictions_single_file)
            all_risk_levels.append(risk_levels_single_file)
            all_data_merged.append(prediction_data_merged) # Append merged data for this file

        return all_predictions, all_risk_levels, all_data_merged # Return lists of predictions, risk levels, and data


    def convert_results_to_json(self, data, predictions, risk_levels, output_filename='results.json'):
        results = []
        # Ensure data is a list of DataFrames when processing individual files
        if not isinstance(data, list):
             data = [data]
             # Adjust predictions and risk_levels structure to match if data was not a list initially
             # This part might need more sophisticated handling depending on how predictions are returned
             # For now, assuming predictions and risk_levels are also lists if data is a list
             if not isinstance(predictions, list):
                 predictions = [predictions]
             if not isinstance(risk_levels, list):
                 risk_levels = [risk_levels]


        for i, df in enumerate(data):
            # Use filename or a unique identifier if available in the DataFrame
            # For now, using index as a placeholder for animal identifier
            animal_id = f'animal_{i+1}' # Simple identifier based on file order

            for index, row in df.iterrows():
                 record_id = f'{animal_id}_record_{index}' # Unique ID for each record
                 record_results = {'record_id': record_id, 'animal_id': animal_id, 'diseases': {}} # Added animal_id to record level
                 for disease in predictions[i]: # Iterate through predictions for the current DataFrame
                     prediction_value = predictions[i][disease][index] if predictions[i][disease] is not None and index < len(predictions[i][disease]) else None
                     risk_level_value = risk_levels[i][disease][index] if risk_levels[i][disease] is not None and index < len(risk_levels[i][disease]) else "Unknown"

                     record_results['diseases'][disease] = {
                         'prediction': float(prediction_value) if prediction_value is not None else None,
                         'risk_level': risk_level_value
                     }
                 results.append(record_results)

        with open(output_filename, 'w') as f:
            json.dump(results, f, indent=4)


    def run_monitoring(self, source, data_path=None, bom_data_path=None):
        data = self.load_data(source, data_path, bom_data_path)
        features = self.prepare_features(data)
        predictions = self.predict_disease_risk(features)
        risk_levels = self.categorize_risk(predictions)
        self.convert_results_to_json(data, predictions, risk_levels)


# Example Usage
if __name__ == "__main__":
    # Define training and prediction data paths
    training_files = ['/content/chicken1.csv', '/content/chicken2.csv', '/content/chicken 4 suffering.csv']
    prediction_files = ['/content/chicken3.csv', '/content/chicken 5 suffering.csv']
    all_chicken_files = ['/content/chicken1.csv', '/content/chicken2.csv', '/content/chicken3.csv', '/content/chicken 4 suffering.csv', '/content/chicken 5 suffering.csv']
    bom_file = '/content/BOM for chicken.csv' # Path to the BOM data
    sprinkler_file = '/content/sprinkler for chicken.csv' # Path to the sprinkler data

    # Example for Chicken Monitoring
    # Model paths are not needed if training is done within the class
    chicken_model_paths = {
        'Avian Influenza': None, # No pre-trained model provided
        'Fowl Typhoid': None,
        'Coccidiosis': None
    }
    chicken_monitor = ChickenMonitoring(chicken_model_paths)

    # Load and merge all chicken data with BOM data
    all_chicken_data_list = chicken_monitor.load_data(source='dataset', data_path=all_chicken_files, bom_data_path=bom_file)

    # Concatenate all chicken data DataFrames into a single DataFrame
    all_chicken_data_merged = pd.concat(all_chicken_data_list, ignore_index=True)

    # Load sprinkler data
    sprinkler_df = pd.read_csv(sprinkler_file)
    sprinkler_df['timestamp'] = pd.to_datetime(sprinkler_df['timestamp'], format='%d-%m-%Y', errors='coerce')

    # Merge the combined chicken data with sprinkler data
    # Assuming 'timestamp' is the common column
    combined_farm_data = pd.merge(all_chicken_data_merged, sprinkler_df, on='timestamp', how='left')

    print("Combined Farm Data:")
    display(combined_farm_data.head())
    display(combined_farm_data.info())

    # Train models using the specified training data and BOM data
    print("Training chicken models...")
    chicken_monitor.train_models(training_files, bom_data_path=bom_file)
    print("Training complete.")

    # Run prediction using the specified prediction data and BOM data
    print("\nRunning prediction on chicken data...")
    predictions, risk_levels, prediction_data_merged = chicken_monitor.predict_with_trained_models(prediction_files, bom_data_path=bom_file)

    # Convert prediction results to JSON
    chicken_monitor.convert_results_to_json(prediction_data_merged, predictions, risk_levels, output_filename='chicken_prediction_results.json')
    print("Chicken prediction results saved to chicken_prediction_results.json")

    # Display prediction results (optional)
    # predictions and risk_levels are now lists of dictionaries
    # prediction_data_merged is a list of DataFrames
    # Need to iterate through the results for display
    print("\nChicken Prediction Results (per file/animal):")
    for i, data_df in enumerate(prediction_data_merged):
        animal_id = f'animal_{i+1}' # Simple identifier
        print(f"\nResults for {animal_id}:")
        # Ensure 'animal_id' exists in the original data_df or add it
        if 'animal_id' not in data_df.columns:
            data_df['animal_id'] = animal_id # Add animal_id for display

        # Create a DataFrame for display for the current file
        results_df_single_file = pd.DataFrame({
            'animal_id': data_df['animal_id'],
            'Avian Influenza_prediction': predictions[i].get('Avian Influenza', [None] * len(data_df)),
            'Avian Influenza_risk': risk_levels[i].get('Avian Influenza', ["Unknown"] * len(data_df)),
            'Fowl Typhoid_prediction': predictions[i].get('Fowl Typhoid', [None] * len(data_df)),
            'Fowl Typhoid_risk': risk_levels[i].get('Fowl Typhoid', ["Unknown"] * len(data_df)),
            'Coccidiosis_prediction': predictions[i].get('Coccidiosis', [None] * len(data_df)),
            'Coccidiosis_risk': risk_levels[i].get('Coccidiosis', ["Unknown"] * len(data_df))
        })
        display(results_df_single_file.head()) # Display head for brevity

Combined Farm Data:


,Day,accelerometer_x,accelerometer_y,accelerometer_z,temperature_x,SpO2,timestamp,temperature_y,humidity,CO2 ppm,CO ppm,NH3 ppm,PM2.5,temperature (°C),SpO2 (%),temperature,activity,operator_1,operator_2,operator_3
0,1,0.421,-1.134,0.726,41.35,93.5,2025-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entry,1.0,0.0,0.0
1,2,-0.724,1.321,-1.173,40.92,95.2,2025-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exit,0.0,1.0,0.0
2,3,1.001,-0.563,1.198,41.14,94.8,2025-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entry,0.0,0.0,1.0
3,4,-1.721,0.981,-0.463,40.75,96.1,2025-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exit,1.0,0.0,0.0
4,5,0.605,-1.782,0.523,41.60,92.9,2025-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entry,0.0,1.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890 entries, 0 to 1889
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Day               1890 non-null   int64         
 1   accelerometer_x   1890 non-null   float64       
 2   accelerometer_y   1890 non-null   float64       
 3   accelerometer_z   1890 non-null   float64       
 4   temperature_x     90 non-null     float64       
 5   SpO2              90 non-null     float64       
 6   timestamp         90 non-null     datetime64[ns]
 7   temperature_y     0 non-null      float64       
 8   humidity          1800 non-null   float64       
 9   CO2 ppm           1800 non-null   float64       
 10  CO ppm            1800 non-null   float64       
 11  NH3 ppm           1800 non-null   float64       
 12  PM2.5             1800 non-null   float64       
 13  temperature (°C)  1800 non-null   float64       
 14  SpO2 (%)          1800 n

None

Training chicken models...
Training data columns after merging BOM: ['Day', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z', 'temperature_x', 'SpO2', 'timestamp', 'temperature_y', 'humidity', 'CO2 ppm', 'CO ppm', 'NH3 ppm', 'PM2.5', 'temperature (°C)', 'SpO2 (%)', 'temperature']
Trained model for Avian Influenza
Trained model for Fowl Typhoid
Trained model for Coccidiosis
Training complete.

Running prediction on chicken data...
Chicken prediction results saved to chicken_prediction_results.json

Chicken Prediction Results (per file/animal):

Results for animal_1:


,animal_id,Avian Influenza_prediction,Avian Influenza_risk,Fowl Typhoid_prediction,Fowl Typhoid_risk,Coccidiosis_prediction,Coccidiosis_risk
0,animal_1,0.028403,low,0.028403,low,0.028403,low
1,animal_1,0.028403,low,0.028403,low,0.028403,low
2,animal_1,0.028403,low,0.028403,low,0.028403,low
3,animal_1,0.028403,low,0.028403,low,0.028403,low
4,animal_1,0.028403,low,0.028403,low,0.028403,low



Results for animal_2:


,animal_id,Avian Influenza_prediction,Avian Influenza_risk,Fowl Typhoid_prediction,Fowl Typhoid_risk,Coccidiosis_prediction,Coccidiosis_risk
0,animal_2,0.998226,high,0.998226,high,0.998226,high
1,animal_2,0.998226,high,0.998226,high,0.998226,high
2,animal_2,0.998226,high,0.998226,high,0.998226,high
3,animal_2,0.998226,high,0.998226,high,0.998226,high
4,animal_2,0.998226,high,0.998226,high,0.998226,high


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import xgboost as xgb
import numpy as np

# Assuming the ChickenMonitoring class and its methods are defined in a previous cell
# and the training_files and bom_file variables are defined.

# Re-load the training data with BOM data for evaluation
chicken_monitor_eval = ChickenMonitoring({}) # Create a new instance, no pre-trained models needed for training here
training_data_eval_list = chicken_monitor_eval.load_data(source='dataset', data_path=training_files, bom_data_path=bom_file)

# Concatenate the list of DataFrames into a single DataFrame for evaluation
if isinstance(training_data_eval_list, list):
    training_data_eval = pd.concat(training_data_eval_list, ignore_index=True)
else:
    training_data_eval = training_data_eval_list


# Prepare features
try:
    features_eval = chicken_monitor_eval.prepare_features(training_data_eval)
except ValueError as e:
    print(f"Error preparing features for evaluation: {e}")
    features_eval = pd.DataFrame() # Create an empty DataFrame to avoid errors

if not features_eval.empty:
    # Create dummy target variables
    # Ensure required columns for dummy target creation exist
    temperature_col_for_target = None
    if 'temperature (°C)' in training_data_eval.columns:
         temperature_col_for_target = 'temperature (°C)'
    elif 'temperature_y' in training_data_eval.columns:
         temperature_col_for_target = 'temperature_y'
    elif 'temperature_x' in training_data_eval.columns:
         temperature_col_for_target = 'temperature_x'

    if 'activity' not in training_data_eval.columns or temperature_col_for_target is None:
        print("Warning: Cannot create dummy targets for evaluation. Missing 'activity' or a suitable temperature column.")
        # Create dummy targets of zeros with the same length as features_eval if features_eval is not empty
        target_avian_influenza = pd.Series([0] * len(features_eval))
        target_fowl_typhoid = pd.Series([0] * len(features_eval))
        target_coccidiosis = pd.Series([0] * len(features_eval))
    else:
        # Ensure the length of the target matches the length of the features
        if len(training_data_eval) != len(features_eval):
            print(f"Warning: Length of training data ({len(training_data_eval)}) does not match length of features ({len(features_eval)}). Skipping dummy target creation for evaluation.")
            target_avian_influenza = pd.Series([0] * len(features_eval))
            target_fowl_typhoid = pd.Series([0] * len(features_eval))
            target_coccidiosis = pd.Series([0] * len(features_eval))
        else:
            # Dummy target: higher risk if activity is low and temp is high
            target_avian_influenza = ((training_data_eval['activity'] < 0.5) & (training_data_eval[temperature_col_for_target] > 30)).astype(int)
            target_fowl_typhoid = ((training_data_eval['activity'] < 0.6) & (training_data_eval[temperature_col_for_target] > 28)).astype(int) # Slightly different dummy logic
            target_coccidiosis = ((training_data_eval['activity'] < 0.4) & (training_data_eval[temperature_col_for_target] > 32)).astype(int) # Slightly different dummy logic


    # Dictionary of dummy targets
    dummy_targets = {
        'Avian Influenza': target_avian_influenza,
        'Fowl Typhoid': target_fowl_typhoid,
        'Coccidiosis': target_coccidiosis
    }

    # Split data into training and testing sets
    # We need to split features and each target separately to ensure consistency
    X_train, X_test, y_train_avian, y_test_avian, y_train_fowl, y_test_fowl, y_train_cocc, y_test_cocc = train_test_split(
        features_eval,
        dummy_targets['Avian Influenza'],
        dummy_targets['Fowl Typhoid'],
        dummy_targets['Coccidiosis'],
        test_size=0.2,
        random_state=42
    )

    # Train a model for each disease on the training set
    trained_models_eval = {}
    for disease in dummy_targets.keys():
        print(f"\nTraining model for {disease} for evaluation...")
        # Get the corresponding training target
        if disease == 'Avian Influenza':
            y_train = y_train_avian
        elif disease == 'Fowl Typhoid':
            y_train = y_train_fowl
        elif disease == 'Coccidiosis':
            y_train = y_train_cocc
        else:
            continue # Skip if disease not in our dummy targets

        if X_train.empty or y_train.empty or y_train.isnull().all():
             print(f"Warning: Training data or target is empty/invalid for {disease}. Skipping training for evaluation.")
             continue

        dtrain = xgb.DMatrix(X_train, label=y_train)
        model = xgb.train({}, dtrain, num_boost_round=10) # Simple training
        trained_models_eval[disease] = model
        print(f"Trained model for {disease} for evaluation.")


    # Evaluate each trained model on the testing set
    print("\nModel Accuracy on Test Set (based on dummy targets):")
    for disease, model in trained_models_eval.items():
        # Get the corresponding testing target
        if disease == 'Avian Influenza':
            y_test = y_test_avian
        elif disease == 'Fowl Typhoid':
            y_test = y_test_fowl
        elif disease == 'Coccidiosis':
            y_test = y_test_cocc
        else:
            continue # Skip if disease not in our dummy targets

        if X_test.empty or y_test.empty or y_test.isnull().all():
             print(f"Warning: Test data or target is empty/invalid for {disease}. Skipping evaluation.")
             continue

        dtest = xgb.DMatrix(X_test)
        predictions = model.predict(dtest)

        # For accuracy, we need to convert predictions to binary (0 or 1) based on a threshold
        # Since our dummy target is binary, we can use a threshold like 0.5 for prediction
        binary_predictions = (predictions > 0.5).astype(int)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, binary_predictions)
        print(f"{disease} Accuracy: {accuracy:.4f}")

else:
    print("Feature preparation failed. Cannot calculate model accuracy.")
    ## Note: Due to less data , the model is geeting overfit


Training model for Avian Influenza for evaluation...
Trained model for Avian Influenza for evaluation.

Training model for Fowl Typhoid for evaluation...
Trained model for Fowl Typhoid for evaluation.

Training model for Coccidiosis for evaluation...
Trained model for Coccidiosis for evaluation.

Model Accuracy on Test Set (based on dummy targets):
Avian Influenza Accuracy: 1.0000
Fowl Typhoid Accuracy: 1.0000
Coccidiosis Accuracy: 1.0000


MODEL-B

In [11]:
import pandas as pd

bom_df = pd.read_csv('/content/BOM for chicken.csv')
sprinkler_df = pd.read_csv('/content/sprinkler for chicken.csv')

print("BOM Data:")
display(bom_df.head())
print("\nBOM Data Types:")
display(bom_df.info())

print("\nSprinkler Data:")
display(sprinkler_df.head())
print("\nSprinkler Data Types:")
display(sprinkler_df.info())

BOM Data:


,timestamp,temperature,humidity,CO2 ppm,CO ppm,NH3 ppm,PM2.5
0,01-01-2025,33.5,68,550,5,10,35
1,02-01-2025,34.0,66,600,4,12,38
2,03-01-2025,33.2,69,520,6,9,34
3,04-01-2025,32.8,70,480,5,8,30
4,05-01-2025,33.9,65,570,7,11,37



BOM Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    30 non-null     object 
 1   temperature  30 non-null     float64
 2   humidity     30 non-null     int64  
 3   CO2 ppm      30 non-null     int64  
 4   CO ppm       30 non-null     int64  
 5   NH3 ppm      30 non-null     int64  
 6   PM2.5        30 non-null     int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 1.8+ KB


None


Sprinkler Data:


,timestamp,activity,operator_1,operator_2,operator_3
0,01-01-2025,entry,1,0,0
1,02-01-2025,exit,0,1,0
2,03-01-2025,entry,0,0,1
3,04-01-2025,exit,1,0,0
4,05-01-2025,entry,0,1,0



Sprinkler Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   timestamp   30 non-null     object
 1   activity    30 non-null     object
 2   operator_1  30 non-null     int64 
 3   operator_2  30 non-null     int64 
 4   operator_3  30 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.3+ KB


None

## Data preprocessing

### Subtask:
Clean and preprocess the loaded BOM and sprinkler data. This may involve handling missing values, converting data types, and potentially aggregating data to a farm level if it's currently at a more granular level.


**Reasoning**:
Convert the 'timestamp' columns to datetime objects and handle potential missing values using forward fill. Then check the data types again.



In [4]:
bom_df['timestamp'] = pd.to_datetime(bom_df['timestamp'], format='%d-%m-%Y')
sprinkler_df['timestamp'] = pd.to_datetime(sprinkler_df['timestamp'], format='%d-%m-%Y')

bom_df.fillna(method='ffill', inplace=True)
sprinkler_df.fillna(method='ffill', inplace=True)

print("BOM Data Types after cleaning:")
display(bom_df.info())
print("\nSprinkler Data Types after cleaning:")
display(sprinkler_df.info())

BOM Data Types after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    30 non-null     datetime64[ns]
 1   temperature  30 non-null     float64       
 2   humidity     30 non-null     int64         
 3   CO2 ppm      30 non-null     int64         
 4   CO ppm       30 non-null     int64         
 5   NH3 ppm      30 non-null     int64         
 6   PM2.5        30 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 1.8 KB


/tmp/ipython-input-1801787590.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  bom_df.fillna(method='ffill', inplace=True)
/tmp/ipython-input-1801787590.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sprinkler_df.fillna(method='ffill', inplace=True)


None


Sprinkler Data Types after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   30 non-null     datetime64[ns]
 1   activity    30 non-null     object        
 2   operator_1  30 non-null     int64         
 3   operator_2  30 non-null     int64         
 4   operator_3  30 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.3+ KB


None

## Feature engineering

### Subtask:
Create relevant features for farm-level risk prediction using the BOM and sprinkler data. This could include metrics like average temperature, humidity, water usage, or correlations between these factors.


**Reasoning**:
Calculate daily aggregate features from the BOM and sprinkler data and merge them for farm-level risk prediction.



In [12]:
daily_bom_features = bom_df.groupby('timestamp').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'CO2 ppm': 'mean',
    'CO ppm': 'mean',
    'NH3 ppm': 'mean',
    'PM2.5': 'mean'
}).reset_index()

sprinkler_df['sprinkler_activity_count'] = sprinkler_df['activity'].apply(lambda x: 1 if x == 'entry' else 0)
sprinkler_df['total_operator_activity'] = sprinkler_df[['operator_1', 'operator_2', 'operator_3']].sum(axis=1)

daily_sprinkler_features = sprinkler_df.groupby('timestamp').agg({
    'sprinkler_activity_count': 'sum',
    'total_operator_activity': 'sum'
}).reset_index()

farm_features_df = pd.merge(daily_bom_features, daily_sprinkler_features, on='timestamp', how='left')

# Calculate interaction terms/ratios if relevant
# Example: Temperature-Humidity Index (THI)
farm_features_df['THI'] = (1.8 * farm_features_df['temperature']) + 32 - (0.55 * (1 - farm_features_df['humidity'] / 100) * (1.8 * farm_features_df['temperature'] - 26))

# Example: Sprinkler activity per operator
# Avoid division by zero if no operator activity
farm_features_df['sprinkler_per_operator'] = farm_features_df.apply(
    lambda row: row['sprinkler_activity_count'] / row['total_operator_activity'] if row['total_operator_activity'] > 0 else 0,
    axis=1
)

print("Farm-level features:")
display(farm_features_df.head())
display(farm_features_df.info())

Farm-level features:


,timestamp,temperature,humidity,CO2 ppm,CO ppm,NH3 ppm,PM2.5,sprinkler_activity_count,total_operator_activity,THI,sprinkler_per_operator
0,01-01-2025,33.5,68.0,550.0,5.0,10.0,35.0,1,1,86.26320,1.0
1,02-01-2025,34.0,66.0,600.0,4.0,12.0,38.0,0,1,86.61760,0.0
2,03-01-2025,33.2,69.0,520.0,6.0,9.0,34.0,1,1,86.00392,1.0
3,04-01-2025,32.8,70.0,480.0,5.0,8.0,30.0,0,1,85.58840,0.0
4,05-01-2025,33.9,65.0,570.0,7.0,11.0,37.0,1,1,86.27865,1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 30 non-null     object 
 1   temperature               30 non-null     float64
 2   humidity                  30 non-null     float64
 3   CO2 ppm                   30 non-null     float64
 4   CO ppm                    30 non-null     float64
 5   NH3 ppm                   30 non-null     float64
 6   PM2.5                     30 non-null     float64
 7   sprinkler_activity_count  30 non-null     int64  
 8   total_operator_activity   30 non-null     int64  
 9   THI                       30 non-null     float64
 10  sprinkler_per_operator    30 non-null     float64
dtypes: float64(8), int64(2), object(1)
memory usage: 2.7+ KB


None

## Data integration

### Subtask:
Integrate the processed BOM and sprinkler data with any relevant aggregated individual chicken data (e.g., overall farm activity levels, average temperature of chickens).


## Model development

### Subtask:
Develop a model (or adapt an existing one) for predicting farm-level chicken health risk based on the engineered features. This might be a regression model to predict a risk score or a classification model to predict risk categories.


**Reasoning**:
Prepare the features and target variable for the model training, then instantiate and train the XGBoost Regressor model.



In [13]:
import xgboost as xgb
import numpy as np

# Assume a target variable 'risk_score' exists for demonstration
# Create a dummy 'risk_score' column if it doesn't exist for the purpose of this subtask
if 'risk_score' not in farm_features_df.columns:
    # Generate some dummy risk scores based on THI for demonstration
    # Higher THI generally implies higher heat stress risk
    # This is a simplified example and should be replaced with actual risk scores
    farm_features_df['risk_score'] = farm_features_df['THI'] * 0.5 + (farm_features_df['sprinkler_per_operator'] > 0).astype(int) * 5

# Select features (excluding 'timestamp' and 'risk_score') and target
features = farm_features_df.drop(columns=['timestamp', 'risk_score'])
target = farm_features_df['risk_score']

# Instantiate the XGBoost Regressor model
farm_risk_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)

# Train the model
farm_risk_model.fit(features, target)

print("Trained XGBoost Regressor model:")
print(farm_risk_model)

Trained XGBoost Regressor model:
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)


## Risk score calculation

### Subtask:
Define and calculate a farm-level risk score based on the model's output.


**Reasoning**:
Use the trained `farm_risk_model` to predict the risk score for the `features` DataFrame and store the predictions in a new column 'predicted_risk_score' in the `farm_features_df` DataFrame, then display the updated DataFrame.



In [ ]:
farm_features_df['predicted_risk_score'] = farm_risk_model.predict(features)
display(farm_features_df)

,timestamp,temperature,humidity,CO2 ppm,CO ppm,NH3 ppm,PM2.5,sprinkler_activity_count,total_operator_activity,THI,sprinkler_per_operator,risk_score,predicted_risk_score
0,2025-01-01,33.5,68.0,550.0,5.0,10.0,35.0,1,1,86.26320,1.0,48.131600,48.131199
1,2025-01-02,34.0,66.0,600.0,4.0,12.0,38.0,0,1,86.61760,0.0,43.308800,43.308453
2,2025-01-03,33.2,69.0,520.0,6.0,9.0,34.0,1,1,86.00392,1.0,48.001960,48.001759
3,2025-01-04,32.8,70.0,480.0,5.0,8.0,30.0,0,1,85.58840,0.0,42.794200,42.795631
4,2025-01-05,33.9,65.0,570.0,7.0,11.0,37.0,1,1,86.27865,1.0,48.139325,48.138866
5,2025-01-06,33.7,67.0,590.0,6.0,10.0,36.0,0,1,86.36921,0.0,43.184605,43.184643
6,2025-01-07,33.3,68.0,560.0,5.0,9.0,33.0,1,1,85.96656,1.0,47.983280,47.983204
7,2025-01-08,34.1,64.0,605.0,4.0,12.0,39.0,0,1,86.37476,0.0,43.187380,43.187332
8,2025-01-09,32.9,70.0,530.0,6.0,8.0,32.0,1,1,85.73870,1.0,47.869350,47.869324
9,2025-01-10,33.4,67.0,575.0,5.0,10.0,35.0,0,1,85.92722,0.0,42.963610,42.963837


In [14]:
import xgboost as xgb
import numpy as np

def calculate_farm_risk(bom_data_path, sprinkler_data_path):
    """
    Calculates the farm-level chicken health risk score.

    Args:
        bom_data_path (str): Path to the BOM data CSV file.
        sprinkler_data_path (str): Path to the sprinkler data CSV file.

    Returns:
        pd.DataFrame: DataFrame containing the original data and predicted risk scores.
    """
    # 1. Load Data
    bom_df = pd.read_csv(bom_data_path)
    sprinkler_df = pd.read_csv(sprinkler_data_path)

    # 2. Preprocessing
    bom_df['timestamp'] = pd.to_datetime(bom_df['timestamp'], format='%d-%m-%Y')
    sprinkler_df['timestamp'] = pd.to_datetime(sprinkler_df['timestamp'], format='%d-%m-%Y')

    bom_df.fillna(method='ffill', inplace=True)
    sprinkler_df.fillna(method='ffill', inplace=True)

    # 3. Feature Engineering
    daily_bom_features = bom_df.groupby('timestamp').agg({
        'temperature': 'mean',
        'humidity': 'mean',
        'CO2 ppm': 'mean',
        'CO ppm': 'mean',
        'NH3 ppm': 'mean',
        'PM2.5': 'mean'
    }).reset_index()

    sprinkler_df['sprinkler_activity_count'] = sprinkler_df['activity'].apply(lambda x: 1 if x == 'entry' else 0)
    sprinkler_df['total_operator_activity'] = sprinkler_df[['operator_1', 'operator_2', 'operator_3']].sum(axis=1)

    daily_sprinkler_features = sprinkler_df.groupby('timestamp').agg({
        'sprinkler_activity_count': 'sum',
        'total_operator_activity': 'sum'
    }).reset_index()

    farm_features_df = pd.merge(daily_bom_features, daily_sprinkler_features, on='timestamp', how='left')

    # Calculate interaction terms/ratios if relevant
    # Example: Temperature-Humidity Index (THI)
    farm_features_df['THI'] = (1.8 * farm_features_df['temperature']) + 32 - (0.55 * (1 - farm_features_df['humidity'] / 100) * (1.8 * farm_features_df['temperature'] - 26))

    # Example: Sprinkler activity per operator
    # Avoid division by zero if no operator activity
    farm_features_df['sprinkler_per_operator'] = farm_features_df.apply(
        lambda row: row['sprinkler_activity_count'] / row['total_operator_activity'] if row['total_operator_activity'] > 0 else 0,
        axis=1
    )

    # 4. Model Training (Retrain for simplicity in this example)
    # Assume a target variable 'risk_score' exists for demonstration
    if 'risk_score' not in farm_features_df.columns:
        # Generate some dummy risk scores based on THI for demonstration
        farm_features_df['risk_score'] = farm_features_df['THI'] * 0.5 + (farm_features_df['sprinkler_per_operator'] > 0).astype(int) * 5

    features = farm_features_df.drop(columns=['timestamp', 'risk_score'])
    target = farm_features_df['risk_score']

    farm_risk_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
    farm_risk_model.fit(features, target)

    # 5. Prediction
    farm_features_df['predicted_risk_score'] = farm_risk_model.predict(features)

    # 6. Return results
    return farm_features_df

# Example usage of the new function
farm_risk_results = calculate_farm_risk('/content/BOM for chicken.csv', '/content/sprinkler for chicken.csv')
print("\nFarm Risk Calculation Results:")
display(farm_risk_results.head())


Farm Risk Calculation Results:


/tmp/ipython-input-2189076398.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  bom_df.fillna(method='ffill', inplace=True)
/tmp/ipython-input-2189076398.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sprinkler_df.fillna(method='ffill', inplace=True)


,timestamp,temperature,humidity,CO2 ppm,CO ppm,NH3 ppm,PM2.5,sprinkler_activity_count,total_operator_activity,THI,sprinkler_per_operator,risk_score,predicted_risk_score
0,2025-01-01,33.5,68.0,550.0,5.0,10.0,35.0,1,1,86.26320,1.0,48.131600,48.131199
1,2025-01-02,34.0,66.0,600.0,4.0,12.0,38.0,0,1,86.61760,0.0,43.308800,43.308453
2,2025-01-03,33.2,69.0,520.0,6.0,9.0,34.0,1,1,86.00392,1.0,48.001960,48.001759
3,2025-01-04,32.8,70.0,480.0,5.0,8.0,30.0,0,1,85.58840,0.0,42.794200,42.795631
4,2025-01-05,33.9,65.0,570.0,7.0,11.0,37.0,1,1,86.27865,1.0,48.139325,48.138866


In [15]:
display(farm_risk_results[['timestamp', 'predicted_risk_score']])

,timestamp,predicted_risk_score
0,2025-01-01,48.131199
1,2025-01-02,43.308453
2,2025-01-03,48.001759
3,2025-01-04,42.795631
4,2025-01-05,48.138866
5,2025-01-06,43.184643
6,2025-01-07,47.983204
7,2025-01-08,43.187332
8,2025-01-09,47.869324
9,2025-01-10,42.963837


## Summary:

### Data Analysis Key Findings

*   The BOM and sprinkler data for chickens were successfully loaded and preprocessed, including converting timestamps to datetime objects and filling missing values using the forward-fill method.
*   Daily aggregated features were engineered from the BOM data (mean temperature, humidity, CO2, CO, NH3, PM2.5) and sprinkler data (total sprinkler activity count, total operator activity).
*   Derived features like the Temperature-Humidity Index (THI) and sprinkler activity per operator were calculated and added to the merged daily feature set.
*   An XGBoost Regressor model was trained on the engineered features to predict a farm-level risk score. A dummy risk score was created based on THI and sprinkler activity for demonstration purposes as no real target variable was available.
*   The trained model successfully predicted risk scores for the provided data.
*   The entire risk calculation logic, from data loading to prediction, was encapsulated within a single function `calculate_farm_risk` for reusability.
*   The calculated predicted risk scores vary over the analyzed period, generally oscillating between approximately 42-43 and 47-48 based on the dummy target variable.


